# Семинар 10: генераторы, итераторы, оператор yield

### Глава 0: нерасказанное про классы

Множественное наследование (+ миксины)

In [ ]:
from dataclasses import dataclass

@dataclass
class EmailMixin:
    email: str


@dataclass
class BasePerson:
    name: str
    age: int


@dataclass
class Person(EmailMixin, BasePerson):
    def __str__(self):
        return f"{self.name}, age {self.age}, email {self.email}"

In [ ]:
me = Person(name="Tema", age=22, email="astreltsov@hse.ru")

print(me)

Обращение к родителю через super()

In [ ]:
from dataclasses import dataclass

@dataclass
class BasePerson:
    name: str
    age: int

    def __str__(self):
        return f"{self.name}, age {self.age}"


@dataclass
class Person(BasePerson):
    def __str__(self):
        print("calling super")
        return super().__str__()

In [ ]:
me = Person(name="Tema", age=22)

print(me)

**Вопрос:** как будет работать если наследование множественное?

### Глава 1: генераторы

Генераторы, это "ленивые" функции, возвращающие значения on demand, когда требуется, например, распаковать их в цикле.

Пример: числа Фибоначчи:

In [ ]:
def generate_fib(max_number):
    fib_1, fib_2 = 1, 1
    yield fib_1  # <---- волшебное слово, чтобы выдать очередное число, но не выходить из функции
    yield fib_2

    for _ in range(2, max_number):
        fib_1, fib_2 = fib_2, fib_1 + fib_2
        yield fib_2

In [ ]:
fibs = generate_fib(10)

print(next(fibs))
print(next(fibs))
print(next(fibs))
print(next(fibs))

In [ ]:
fibs = generate_fib(10)
for x in fibs:
    print(x)

In [ ]:
for x in fibs:  # второй раз уже не выведет ничего
    print(x)

In [ ]:
next(fibs)  # а next выдаст ошибку

А если рекурсия?

In [ ]:
def traverse_dict(d):
    if not isinstance(d, dict):
        yield d
    else:
        for v in d.values():
            yield from traverse_dict(v)

In [ ]:
d = {
    "one": {
        "two": {
            "three": "four",
            "five": "six"
        }
    },
    "seven": "eight",
}

for x in traverse_dict(d):
    print(x)

### Глава 2: итераторы

In [ ]:
collection = [1, 2, 3, 4, 5]

list_iter = iter(collection)

In [ ]:
print(next(list_iter))
print(next(list_iter))
print(next(list_iter))
print(next(list_iter))
print(next(list_iter))

In [ ]:
print(next(list_iter))

In [ ]:
for x in enumerate(collection):  # <---- это тоже итератор
    print(x)

In [ ]:
f = open("input.txt")  # <--- и это тоже!

f.readline()  # <--- а это фактически его next

In [ ]:
class FibonacciIterator:
    def __init__(self, max_number):
        self.prev = 1
        self.cur = 1
        self.num = 0
        self.max_number = max_number

    def __next__(self):
        if self.num == self.max_number:
            raise StopIteration

        result = self.prev
        self.prev, self.cur = self.cur, self.prev + self.cur
        self.num += 1
        return result

    def __iter__(self):
        return self

In [ ]:
fib_iter = FibonacciIterator(10)

for x in fib_iter:
    print(x)

In [ ]:
class SquareIterator:
    def __init__(self, initial_number):
        # Здесь хранится промежуточное значение
        self.number_to_square = initial_number

    def __next__(self):
        # Здесь мы обновляем значение и возвращаем результат
        self.number_to_square = self.number_to_square ** 2
        return self.number_to_square

    def __iter__(self):
        return self

In [ ]:
sq_iter = SquareIterator(2)

print(next(sq_iter))
print(next(sq_iter))
print(next(sq_iter))
print(next(sq_iter))

Важные моменты:

1) Генераторы -- это тоже итераторы
2) Любой объект, по которому можно сделать for (list, str, dict, set и тд) -- реализует протокол итератора

### Задание 1

написать простой итератор, по которому можно проходиться неограниченное количество раз, при этом допускается хранение элементов, но лишь столько, сколько максимально мы успели просмотреть

### Задание 2

написать свой генератор chain, который принимает на себя через * список коллекций, а возвращает лениво их итеративную склейку, например:

```
chain([1, 2, 3], {"a", "b", "c"}) -> 1, 2, 3, a, b, c
```

### Задание 3

написать свой генератор flatten, принимающий коллекцию с вложенными iterable-сущностями, а возвращающую лениво (сплющенный список), например:

```
[[1, 2, 3], [4, [5, 6]]] -> [1, 2, 3, 4, 5, 6]
```

Для удобства проверять на итерируемость можно через

```
from collections.abc import Iterable
...

if isinstance(x, Iterable):
    ...
```

In [ ]:
from collections.abc import Iterable

# a = [1, 2, 3]
# isinstance(a, Iterable)

def flatten(iterable: Iterable):
    ...